In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from statistics import median, mean
from matplotlib import pyplot as plt
import numpy as np
import json
from tqdm import tqdm
from glob import glob
import os
import sys
sys.path.insert(0,"/study/mrphys/skunkworks/kk/mriUnet")
from fullmodel import fullModel
from unet import UNet
from torchvision import transforms
from torch.utils.data import Dataset
from sklearn.model_selection import KFold as kf
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
from mriDataset import mriSliceDataset
from smoothing import GaussianSmoothing

In [2]:
b = 1
nchans = 10
f = 16
s = 128
device = 6

model = UNet(
    nchans,
    nchans,
    f_maps=f,
    layer_order=['separable convolution', 'relu'],
    depth=4,
    layer_growth=2.0,
    residual=True,
    complex_input=True,
    complex_kernel=True,
    ndims=3,
    padding=1
)

data = torch.randn(b,10,s,s,s) + torch.randn(b,10,s,s,s)*1J

model = model.to(device)
data = data.to(device)

predictions = model(data)

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [3]:
def complex_2d_ssim(pred, true):
    ssim_loss_real = (1-ssim(pred.real, true.real, data_range=1, size_average=True))
    ssim_loss_imag = (1-ssim(pred.imag, true.imag, data_range=1, size_average=True))
    return ssim_loss_real+ssim_loss_imag

def complex_3d_ssim(pred_3d, true_3d):
    total_loss = 0
    for i in range(s):
        total_loss += complex_2d_ssim(pred_3d[:,:,i], true_3d[:,:,i])
        total_loss += complex_2d_ssim(pred_3d[:,:,:,i], true_3d[:,:,:,i])
        total_loss += complex_2d_ssim(pred_3d[:,:,:,:,i], true_3d[:,:,:,:,i])
    return total_loss
        
loss = complex_3d_ssim(predictions, data)

In [4]:
loss.backward()

In [5]:
!nvidia-smi

Thu May 25 23:45:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                 Off| 00000000:01:00.0 Off |                  Off |
| 33%   40C    P2              143W / 260W|  16961MiB / 49152MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--